# Homework 7: Machine Translation
Spencer Hann  
CS 562 | Winter 2019

## Part 1: IBM Model 1

In [1]:
from model1 import Model1

m = Model1()
%time m.train(5)

CPU times: user 7min 21s, sys: 792 ms, total: 7min 22s
Wall time: 7min 22s


In [2]:
print("\nf words translated:")
with open("./data/fwords.txt") as f:
    for word in f:
        word = word.strip()
        tabs = '\t' if len(word) > 6 else '\t\t' # column spacing
        print(word, tabs, m.translate_word(word))


f words translated:
AFFAIRES 	 AFFAIRS
AMENDEMENT 	 AMENDMENT
AUTOCHTONES 	 ABORIGINAL
CHAMBRE 	 HOUSE
CHEF 		 LEADER
COMITÉ 		 COMMITTEE
COMMERCE 	 TRADE
COMMUNES 	 COMMONS
DROITS 		 RIGHTS
DÉCLARATION 	 STATEMENT
DÉFENSE 	 DEFENCE
DÉVELOPPEMENT 	 DEVELOPMENT
ENFANTS 	 CHILDREN
FINANCES 	 FINANCE
FÉDÉRAL 	 FEDERAL
GENS 		 PEOPLE
GOUVERNEMENT 	 GOVERNMENT
GUERRE 		 WAR
HISTOIRE 	 HISTORY
INTERNATIONALE 	 INTERNATIONAL
JUGE 		 JUSTICE
MINISTÈRE 	 DEPARTMENT
MONDE 		 WORLD
NATIONALE 	 NATIONAL
PARLEMENT 	 PARLIAMENT
PAROLE 		 SPEAK
PARTIE 		 PART
PREMIER 	 PRIME
PREMIÈRE 	 FIRST
PROGRAMME 	 PROGRAM
PROJET 		 BILL
PROVINCE 	 PROVINCE
PRÉSIDENT 	 SPEAKER
QUÉBEC 		 QUEBEC
RAISON 		 REASON
RAPPORT 	 REPORT
RESPONSABILITÉ 	 RESPONSIBILITY
RÉGIME 		 PLAN
RÉGION 		 REGION
RÉPONSE 	 ANSWER
SECTEUR 	 SECTOR
SERVICES 	 SERVICES
SOCIÉTÉ 	 SOCIETY
SÉCURITÉ 	 SECURITY
SÉNAT 		 SENATE
SÉNATEUR 	 SENATOR
TRAVAIL 	 WORK
ÉGALEMENT 	 ALSO
ÉTATS-UNIS 	 STATES
ÉTUDE 		 STUDY


For this problem, I made a heavy use of the `collections.defaultdict` object. At first I relied too heavily on the convience of this data structure, and my memory would overflow long before making it through the entire data set. For my maximization step I was looping over every possible pair of words between the source and target vocabularies, not only did this require a lot of memory, most of that memory was taken up by dictionary entries that simply held `0`. Initially I tried putting a lower threshold on whether or not some count would "count". For example, I would only store some probability $p(w_t | w_s)$ if $count(w_t | w_s) > threshold$. I experiment with different thresholds, but even when it was quite high, I still would run out of RAM. I realized that the line of code `if count[e][f] > 0:` would still cause the `defaultdict` to create a new entry for `[e][f]`. When I changed that line to `if f in count[e]:`, my program began to run very smoothly. I included some other optimizations as well, like, rather than using a generator in one of the inner loops, I create a list using the generator at the top of the function so that the list can be re-used with out the sentence pairs having to be re-generated every iteration.  

After the first iteration, pretty much everything translates to "`THE`". My model begins to produce relatively good translations in as few as 2 iterations, but there are still some hold-outs. For example, "`PRÉSIDENT`" still maps to "`THE`" until iteration 4, at which point it maps to "`SPEAKER`". It seems to learn most of what it can learn in the first several iterations, but may require more to learn words beyond these 50.  

I trained it up to 25 iterations, nothing much changes after 4 iterations ("`ÉTATS-UNIS`" now maps to "`UNITED`" instead of "`STATES`"), and "`PRÉSIDENT`" still maps to "`SPEAKER`", making me think that, at least for this corpus, in these contexts "`SPEAKER`" is not a bad translation. This isn't totally surprising since these documents are governmental procedings from Canada, which has a Prime Minister instead of a President, unlike France and the US.

In [3]:
%time m.train(20) # additional 20 iterations on existing model pre-trained for 5

CPU times: user 31min 57s, sys: 1.75 s, total: 31min 59s
Wall time: 31min 59s


In [4]:
print("\nf words translated:")
with open("./data/fwords.txt") as f:
    for word in f:
        word = word.strip()
        tabs = '\t' if len(word) > 6 else '\t\t' # column spacing
        print(word, tabs, m.translate_word(word))


f words translated:
AFFAIRES 	 AFFAIRS
AMENDEMENT 	 AMENDMENT
AUTOCHTONES 	 ABORIGINAL
CHAMBRE 	 HOUSE
CHEF 		 LEADER
COMITÉ 		 COMMITTEE
COMMERCE 	 TRADE
COMMUNES 	 COMMONS
DROITS 		 RIGHTS
DÉCLARATION 	 STATEMENT
DÉFENSE 	 DEFENCE
DÉVELOPPEMENT 	 DEVELOPMENT
ENFANTS 	 CHILDREN
FINANCES 	 FINANCE
FÉDÉRAL 	 FEDERAL
GENS 		 PEOPLE
GOUVERNEMENT 	 GOVERNMENT
GUERRE 		 WAR
HISTOIRE 	 HISTORY
INTERNATIONALE 	 INTERNATIONAL
JUGE 		 JUSTICE
MINISTÈRE 	 DEPARTMENT
MONDE 		 WORLD
NATIONALE 	 NATIONAL
PARLEMENT 	 PARLIAMENT
PAROLE 		 SPEAK
PARTIE 		 PART
PREMIER 	 PRIME
PREMIÈRE 	 FIRST
PROGRAMME 	 PROGRAM
PROJET 		 BILL
PROVINCE 	 PROVINCE
PRÉSIDENT 	 SPEAKER
QUÉBEC 		 QUEBEC
RAISON 		 REASON
RAPPORT 	 REPORT
RESPONSABILITÉ 	 RESPONSIBILITY
RÉGIME 		 PLAN
RÉGION 		 REGION
RÉPONSE 	 ANSWER
SECTEUR 	 SECTOR
SERVICES 	 SERVICES
SOCIÉTÉ 	 SOCIETY
SÉCURITÉ 	 SECURITY
SÉNAT 		 SENATE
SÉNATEUR 	 SENATOR
TRAVAIL 	 WORK
ÉGALEMENT 	 ALSO
ÉTATS-UNIS 	 UNITED
ÉTUDE 		 STUDY


## Part 2: BLEU

In [5]:
from bleu import BLEU, bitext

bleu_google = list()
bleu_systran = list()

with open("data/gtranslate.tok") as gtran, open("data/e.tok") as ref:
    for pair in bitext(gtran, ref):
        bleu_google.append(BLEU(*pair))

with open("data/systran.tok") as systran, open("data/e.tok") as ref:
    for pair in bitext(systran, ref):
        bleu_systran.append(BLEU(*pair))

print("  Google Translate", "  SYSTRAN")
for i,scores in enumerate(zip(bleu_google,bleu_systran)):
    print(i,*scores)

  Google Translate   SYSTRAN
0 0.8429437482062828 0.6919161984038511
1 0.8449147212133874 0.7803216053875227
2 0.7580761160995355 0.651501860939786
3 0.7143679962281008 0.8241106303521183
4 0.690348542961804 0.5664218053975602
5 1.0492484809296132 0.9557393754442446
6 0.7983868786305349 0.8139957963337889
7 0.8578928092681435 0.7437868342924722
8 0.8562839466057395 0.69712983741573
9 0.9291564424807219 0.8135647684906822


By simply looking at the scores, it would appear that Google Translate is a significantly better translator. However, when looking at the actual sentences, their perfomances seem to be very similar. The SYSTRAN output seem to be more literal, where the Google Translate output, while not translating words directly from the french source, matches the english reference more closely. For example, where the reference translation reads "...AND IT IS CERTAINLY A BURDEN ON...", Google's translation reads "...AND IT IS A BURDEN FOR...", and SYSTRAN's translation reads, "...AND THAT CONSTITUTES A BURDEN FOR...". As a native English speaker, who does not understand much French, I find the "it is a" phrasing better/simpler/more intuitive than the "constitutes a", but when referencing the original French sentence which reads, "...cela constitue un...", I have to wonder which really is better. How hard should the translator try to interpret meaning over translate literally/faithfully? Obviously this is objective, but it highlights how BLEU is at the mercy of the reference translations used, even if the similarity metric is reliable.  

The main bug I ran into with this function was actually a little silly, but no less of a headache. I originally passed `"data/gtranslate.tok", "data/e.tok"` to the `bitext` function. My output was very strange and also very uniform. I was actually just taking the BLEU score, character-wise, of the strings with `"data/gtranslate.tok"` as the hypothesis and `"data/e.tok"` as the reference. I didn't think about the fact that `bitext` was just taking in strings, not reading anything from file. This is a reason why it can be important to actually read, or at least be sure you understand, the code you use.